## This file helps to Build and test the LLama model and Langchain functions

In [16]:
# Importing packages

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [17]:
#setting up env
load_dotenv()

True

In [18]:
#Building LLM
llm = ChatGroq(
    model='llama3-8b-8192',
    groq_api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.5
    )

In [19]:
#Getting response 
response = llm.invoke("you are a Career advisor ask the user some question with answer options using the following details: Name - Pugazh : Interests: Coding, profession : ML engineer")
print(response)

content="Hello Pugazh! I'm excited to help you explore your career options as a Machine Learning Engineer. Let's get started!\n\nCan you tell me what motivates you most about your work as an ML Engineer?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 44, 'total_tokens': 89, 'completion_time': 0.0375, 'prompt_time': 0.00609325, 'queue_time': 0.020940078, 'total_time': 0.04359325}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None} id='run-51a11494-2318-4b69-b1ae-15ea23e7619c-0' usage_metadata={'input_tokens': 44, 'output_tokens': 45, 'total_tokens': 89}
